### Boids 
Boids is an artificial life simulation originally developed by Craig Reynolds. The aim of the simulation was to replicate the behavior of flocks of birds. Instead of controlling the interactions of an entire flock, however, the Boids simulation only specifies the behavior of each individual bird.

In [ ]:
import numpy as np

In [ ]:
n_boids=20

In [ ]:
window_size=[600,600] #display window limit

In [ ]:
position=np.random.rand(2, n_boids)*250 #random number less than 250
position.astype(int) #convert to integer
print(position)
print(position[:,1]) # print the second column

We set the random velocities within some limits. 

In [ ]:
Lb=np.array([-2,-2]) #lower bound variables
Ub=np.array([2,2]) 
diff=Ub-Lb
print(diff)
velocity=Lb[:, np.newaxis]+np.random.rand(2,n_boids)*diff[:,np.newaxis] 
velocity

`np.newaxis` creates a new index. Print them out to see why

In [ ]:
np.random.rand(2,n_boids)

In [ ]:
diff[:,np.newaxis] 

In [ ]:
position=position+velocity
pos=np.array(position.astype(int)) # we require integer coordinates
pos

We want to check if the position exceeds the window limit, if so, reverse the velocity. Complete the code below.

In [ ]:
velocity[0]=[-1*v if p > window_size[0] else v for p,v in zip(pos[0],velocity[0])]
#velocity[1]= ..  #complete this 
print(velocity)
# check that limits are not exceeded

In [ ]:
velocity[0]=[-1*v if p < 0 else v for p,v in zip(pos[0],velocity[0])]
#velocity[1]=   # complete this
print(velocity)
# check that limits are not exceeded

Create a function that calculates the distance between to position. Use the Euclidean metric to calculate. Complete the code below. Use `np.sqrt` to calculate square root.

In [ ]:
def dist(position,i,j):
    #xdiff = 
    #ydiff = 
    #distance = 
    return distance

### Three Flocking Rules

1. Cohesion
1. Repulsion
1. Alignment

    Cohesion
	PROCEDURE rule1(boid bJ)

		Vector pcJ

		FOR EACH BOID b
			IF b != bJ THEN
				pcJ = pcJ + b.position
			END IF
		END

		pcJ = pcJ / N-1

		RETURN (pcJ - bJ.position) / 100

	END PROCEDURE
    
    Repulsion
    PROCEDURE rule2(boid bJ)

		Vector c = 0;

		FOR EACH BOID b
			IF b != bJ THEN
				IF |b.position - bJ.position| < 100 THEN
					c = c - (b.position - bJ.position)
				END IF
			END IF
		END

		RETURN c

	END PROCEDURE
    
    Alignment
    PROCEDURE rule3(boid bJ)

		Vector pvJ

		FOR EACH BOID b
			IF b != bJ THEN
				pvJ = pvJ + b.velocity
			END IF
		END

		pvJ = pvJ / N-1

		RETURN (pvJ - bJ.velocity) / 8

	END PROCEDURE

All the return vector values should be normalized using `np.linalg.norm`

see http://www.kfish.org/boids/pseudocode.html

In [ ]:
# cohesion
def cohesion(j):
    cx=0
    cy=0
    count=0
    for i in range(n_boids):
        if (j!=i):
            distance=dist(position,i,j)
            if(distance<5):
                count=count+1
                cx=cx+position[0,i]
                cy=cy+position[1,i]
                
    if(count>0):
        cx=cx/count
        cy=cy/count
        
    rx=cx-position[0,j]
    ry=cy-position[1,j]
    norm=np.linalg.norm([rx,ry])
    if(norm>0):
        rx=rx/norm
        ry=ry/norm
    #print(rx,ry)
    return (rx,ry)

In [ ]:
# repulsion
def repulsion(j):
    px=0
    py=0
    for i in range(n_boids):
        if (j!=i):
            distance=dist(position,i,j)
            if(distance < 10):
                # fill in here
                # file in here
    norm=np.linalg.norm([px,py])            
    if(norm>0):
        px=px/norm
        py=py/norm
    #print(px,py)
    return (px,py)

In [ ]:
# alignment
def alignment(j):
    vx=0
    vy=0
    count=0
    for i in range(n_boids):
        if(j!=i):
            distance=dist(position,i,j)
            if(distance<20):
                count=count+1
                #fill in here
                #fill in here
    if(count>0):
        vx=vx-velocity[0,j]/count
        vy=vy-velocity[1,j]/count
        
    norm=np.linalg.norm([vx,vy])
    if(norm>0):
        vx=vx/norm
        vy=vy/norm
    return (vx,vy)

### Put all in a game loop

In [ ]:
import pygame
pygame.init()

#Try various values until works 
#A=
#B=
#C=

# Set up the drawing window
screen = pygame.display.set_mode(window_size)

# Run until the user asks to quit
running = True
while running:

    # Did the user click the window close button?
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Fill the background with white
    screen.fill((255, 255, 255))
    pos=position.astype(int) # convert to integers
    
        
    for i in range(n_boids):
        pygame.draw.circle(screen, (0, 0, 255), pos[:,i], 5)
    
    if(np.random.random()<0.001):
        A=A*-1
        C=C*-1
    
    for i in range(n_boids): #individual updates
        v1x,v1y=cohesion(i)
        v2x,v2y=repulsion(i)
        v3x,v3y=alignment(i)

        velocity[0,i]+=A*v1x+B*v2x+C*v3x
        velocity[1,i]+=A*v1y+B*v2y+C*v3y
        
        velocity[0,i]=velocity[0,i]/np.linalg.norm([velocity[0,i],velocity[1,i]])
        velocity[1,i]=velocity[1,i]/np.linalg.norm([velocity[0,i],velocity[1,i]])
        
        position[0,i]+=velocity[0,i]
        position[1,i]+=velocity[1,i]
        #print(position)
    
    
    #check if position exceeds limit
    velocity[0]=[-1*v if p > window_size[0] else v for p,v in zip(position[0],velocity[0])]
    velocity[1]=[-1*v if p > window_size[1] else v for p,v in zip(position[1],velocity[1])]
    velocity[0]=[-1*v if p < 0 else v for p,v in zip(position[0],velocity[0])]
    velocity[1]=[-1*v if p < 0 else v for p,v in zip(position[1],velocity[1])]
    
    position[0]=[1 if p < 0 else p for p in position[0]]
    position[1]=[1 if p < 0 else p for p in position[1]]

    position[0]=[window_size[0]-1 if p > window_size[0] else p for p in position[0]]
    position[1]=[window_size[1]-1 if p > window_size[1] else p for p in position[1]]
    
    # Flip the display
    pygame.display.flip()

# Done! Time to quit.
pygame.display.quit()

### Try it Yourself
1. Change the values A,B and C. Slight changes will change the behavior quite unpredictably